## Pokeapi - CoderHouse
Jupyter Notebook com as execuções de entrega do projeto, respeitando:
1. Coleta de dados de uma API
2. Tratamento desses dados
3. Armazenamento em banco de dados

#### Começamos importando as bibliotecas e funções
1. pandas - Para analise dos dados coletados
2. src.utils - Arquivo com as funções criadas para conexão na API, coleta, tratamento e armazenamento dos dados
3. src.connect_database - Usado para fazer a conexão com o banco de dados.
4. src.config - Onde esta as queries de "input" de dados no banco, assim como as informações de conesão.  

In [1]:
import pandas as pd
from src.utils import *
from src.connect_database import *
from src.config import QUERY_INSERT

### Inicia fazendo a conexão na API
A página inicial traz uma série de URLs para cada Pokemon

In [2]:
pokemon = get_all_pokemons_from_api()
pokemon

'https://pokeapi.co/api/v2/pokemon/1/'

### Com a URL do Pokemon, inicia o tratamento dos dados

Coleta as informações do Pokemon

In [3]:
pokemon_info = get_pokemon_info(pokemon)

client_db = connect_to_mysql()
query_pokemon = QUERY_INSERT['pokemon']
store_pokemon_data(pokemon_info, client_db, query_pokemon)

disconnect_to_mysql(client_db)

df_pokemon_info = pd.DataFrame(pokemon_info)
df_pokemon_info

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


,pokemon_id,name,height,weight
0,1,bulbasaur,7,69


Coleta as informações de habilidades

In [4]:
pokemon_abilities, pokemon_abilities_ids_aux = get_abilities_from_api(pokemon)

client_db = connect_to_mysql()
query_abilities = QUERY_INSERT['abilities']
store_pokemon_data(pokemon_abilities, client_db, query_abilities)

query_pokemon_abilities = QUERY_INSERT['pokemon_abilities']
store_pokemon_data(pokemon_abilities_ids_aux, client_db, query_pokemon_abilities)

disconnect_to_mysql(client_db)

df_pokemon_abilities = pd.DataFrame(pokemon_abilities)
df_pokemon_abilities

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


,abilitie_id,name
0,65,overgrow
1,34,chlorophyll


Coleta as informações dos movimentos

In [5]:
pokemon_moves, pokemon_moves_ids_aux = get_moves_from_api(pokemon)

client_db = connect_to_mysql()
query_moves = QUERY_INSERT['moves']
store_pokemon_data(pokemon_moves, client_db, query_moves)

query_pokemon_moves = QUERY_INSERT['pokemon_moves']
store_pokemon_data(pokemon_moves_ids_aux, client_db, query_pokemon_moves)

disconnect_to_mysql(client_db)

df_pokemon_moves = pd.DataFrame(pokemon_moves)
df_pokemon_moves

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


,move_id,name
0,13,razor-wind
1,14,swords-dance
2,15,cut
3,20,bind
4,22,vine-whip
...,...,...
81,580,grassy-terrain
82,590,confide
83,803,grassy-glide
84,851,tera-blast


# Analise dos dados

In [6]:
client_db = connect_to_mysql()
query = """
SELECT p.pokemon_id AS POKEMON_ID, p.name AS pokemon_name, a.name AS abilitie_name, m.name AS move_name
    FROM pokemon p
    LEFT JOIN pokemon_abilities pa ON p.pokemon_id = pa.pokemon_id
    LEFT JOIN abilities a ON pa.ability_id = a.abilitie_id
    LEFT JOIN pokemon_moves pm ON p.pokemon_id = pm.pokemon_id
    LEFT JOIN moves m ON pm.move_id = m.move_id
    GROUP BY p.pokemon_id,p.name, a.name, m.name
    ORDER BY p.name;
"""

df_select = pd.read_sql_query(query, client_db)
disconnect_to_mysql(client_db)

df_select

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


/tmp/ipykernel_25415/2059514234.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_select = pd.read_sql_query(query, client_db)


,POKEMON_ID,pokemon_name,abilitie_name,move_name
0,1,bulbasaur,overgrow,synthesis
1,1,bulbasaur,overgrow,weather-ball
2,1,bulbasaur,overgrow,secret-power
3,1,bulbasaur,overgrow,knock-off
4,1,bulbasaur,overgrow,ingrain
...,...,...,...,...
167,1,bulbasaur,chlorophyll,light-screen
168,1,bulbasaur,chlorophyll,defense-curl
169,1,bulbasaur,chlorophyll,double-team
170,1,bulbasaur,chlorophyll,mimic
